# Solution

This notebook shows the procedure we used to generate the final submission for the challenge. We first used the median of 146 models (selected by leaving 1 drug out for NK cells) as the base prediction. This model was trained only on the top 64 most variable genes (sorted by variance), and was used to predict the ~18K signed log-pvalues.

We then trained the same NN on a subset of the data. This produced again 146 models that we used to generate a median prediction only on that subset of the data. 

Finally, we replaced the base predictions with the for the final submission.

In [1]:
import scape
import pandas as pd
import numpy as np

scape.__version__, pd.__version__, np.__version__

('0.1.0', '2.1.3', '1.26.2')

In [2]:
df_de = scape.io.load_slogpvals("_data/de_train.parquet")
df_lfc = scape.io.load_lfc("_data/lfc_train.parquet")

# Make sure rows/columns are in the same order
df_lfc = df_lfc.loc[df_de.index, df_de.columns]
df_de.shape, df_lfc.shape

((614, 18211), (614, 18211))

In [3]:
# We select only a subset of the genes for the model (top most variant genes)
n_genes = 64
top_genes = scape.util.select_top_variable([df_de], k=n_genes)
#scm = scape.model.create_default_model(n_genes, df_de, df_lfc)
#scape.util.plot(scm.model, show_shapes=True)

## Base predictions

In [4]:
cell = "NK cells"
drugs = df_de.loc[df_de.index.get_level_values("cell_type") == cell].index.get_level_values("sm_name").unique().tolist()
len(drugs)

146

In [5]:
df_id_map = pd.read_csv("_data/id_map.zip")
df_sub = pd.read_csv("_data/sample_submission.zip", index_col = 0)
df_sub_ix = df_id_map.set_index(["cell_type", "sm_name"])
df_sub_ix

id
cell_type     sm_name                                                
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...    0
              ABT-199 (GDC-0199)                                    1
              ABT737                                                2
              AMD-070 (hydrochloride)                               3
              AT 7867                                               4
...                                                               ...
Myeloid cells Vandetanib                                          250
              Vanoxerine                                          251
              Vardenafil                                          252
              Vorinostat                                          253
              YK 4-279                                            254

[255 rows x 1 columns]

In [6]:
base_predictions = []
for i, d in enumerate(drugs):
    print(i, d)
    scm = scape.model.create_default_model(n_genes, df_de, df_lfc)
    result = scm.train(
        val_cells=[cell], 
        val_drugs=[d],
        input_columns=top_genes,
        epochs=300,
        output_folder="_models",
        config_file_name="config.pkl",
        model_file_name=f"drug{i}.keras",
        baselines=["zero", "slogpval_drug"],
    )
    # Collect prediction in the OOF data
    df_pred = scm.predict(df_sub_ix)
    df_pred = df_pred.loc[:, df_sub.columns]
    base_predictions.append(df_pred)

df_sub = pd.DataFrame(np.median(base_predictions, axis=0), index=df_sub_ix.index, columns=df_sub.columns)
df_sub


0 Clotrimazole

Baseline zero MRRMSE: 0.6085
Baseline slogpval_drug MRRMSE: 0.6606
Model size (MB): 36.76
Model inputs: ['in_slogpval_drug', 'in_lfc_drug', 'in_slogpval_cell_encoder', 'in_lfc_cell_encoder', 'in_slogpval_cell_decoder', 'in_lfc_cell_decoder']
Model will be saved to _models\drug0.keras

Epoch    1: val_loss None -> 0.6019, train loss: 1.3085 (lr=0.00500)
	 > Improvement factor/percentage zero                 (err=0.6085): 1.0109, 1.08%
	 > Improvement factor/percentage slogpval_drug        (err=0.6606): 1.0975, 8.89%
Epoch    2: val_loss 0.6019 -> 0.5870, train loss: 1.2811 (lr=0.00500)
	 > Improvement factor/percentage zero                 (err=0.6085): 1.0366, 3.53%
	 > Improvement factor/percentage slogpval_drug        (err=0.6606): 1.1254, 11.14%
Epoch   16: val_loss 0.5870 -> 0.5661, train loss: 1.1444 (lr=0.00500)
	 > Improvement factor/percentage zero                 (err=0.6085): 1.0749, 6.97%
	 > Improvement factor/percentage slogpval_drug        (err=0.6606): 1.

A1BG  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.032278   
              ABT-199 (GDC-0199)                                 -0.019174   
              ABT737                                              0.323903   
              AMD-070 (hydrochloride)                            -0.099841   
              AT 7867                                             0.099627   
...                                                                    ...   
Myeloid cells Vandetanib                                          0.021890   
              Vanoxerine                                          0.295531   
              Vardenafil                                          0.074475   
              Vorinostat                                          0.505049   
              YK 4-279                                            0.068411   

                                                                  A1BG-AS1  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.151656   
              ABT-199 (GDC-0199)                                  0.026636   
              ABT737                                              0.065766   
              AMD-070 (hydrochloride)                             0.122193   
              AT 7867                                             0.069439   
...                                                                    ...   
Myeloid cells Vandetanib                                         -0.065695   
              Vanoxerine                                         -0.084059   
              Vardenafil                                          0.029403   
              Vorinostat                                          0.182648   
              YK 4-279                                           -0.107688   

                                                                       A2M  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.387460   
              ABT-199 (GDC-0199)                                 -0.013851   
              ABT737                                              0.088764   
              AMD-070 (hydrochloride)                             0.034077   
              AT 7867                                             0.160960   
...                                                                    ...   
Myeloid cells Vandetanib                                         -0.664498   
              Vanoxerine                                         -0.424479   
              Vardenafil                                         -0.055442   
              Vorinostat                                         -2.302280   
              YK 4-279                                           -0.798130   

                                                                   A2M-AS1  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.837589   
              ABT-199 (GDC-0199)                                 -0.032553   
              ABT737                                              0.297251   
              AMD-070 (hydrochloride)                             0.042597   
              AT 7867                                             0.343156   
...                                                                    ...   
Myeloid cells Vandetanib                                         -0.637244   
              Vanoxerine                                         -0.364767   
              Vardenafil                                         -0.303477   
              Vorinostat                                         -0.449509   
              YK 4-279                                           -0.770061   

          

## Enhanced predictions

We selected a subset of the dataset consisting of the top 256 genes and top 60 drugs (sorted by variance). We trained the same model as before on this subset of the data, and used the 146 models to generate a median prediction on this subset of the data. We finally merged the results with the base predictions using a weighted mean (0.80 for the enhaced predictions in the subset of 256 genes and 60 drugs, 0.20 for the base prediction)

In [7]:
sub_drugs = df_sub_ix.index.get_level_values("sm_name").unique().tolist()
len(sub_drugs)

129

In [8]:
min_n_top_drugs = 50
n_genes = 256

# This time, exclude control drugs for the calculation of the top genes, in order to
# introduce more variability in the model
top_genes = scape.util.select_top_variable([df_de], k=n_genes, exclude_controls=True)

In [9]:
df_drug_effects = pd.DataFrame(df_de.T.pow(2).mean().pow(0.5).groupby("sm_name").mean().sort_values(ascending=False), columns=["effect"])
df_drug_effects["effect_norm"] = (df_drug_effects["effect"] / df_drug_effects["effect"].sum())*100
df_drug_effects

,effect,effect_norm
sm_name,,
Belinostat,11.670403,6.522607
Alvocidib,8.718604,4.872841
MLN 2238,7.945529,4.440769
CEP-18770 (Delanzomib),7.204718,4.026728
Oprozomib (ONX 0912),6.818472,3.810855
...,...,...
RN-486,0.534020,0.298465
OSI-930,0.533040,0.297917
K-02288,0.525925,0.293940


In [10]:
top_sub_drugs = df_drug_effects.loc[sub_drugs].sort_values("effect", ascending=False).head(min_n_top_drugs).index.tolist()
len(top_sub_drugs)

50

In [11]:
top_all_drugs = df_drug_effects.head(min_n_top_drugs).index.tolist()
top_drugs = set(top_all_drugs) | set(top_sub_drugs)
len(top_drugs)

60

In [12]:
df_de_c = df_de[df_de.index.get_level_values("sm_name").isin(top_drugs)]
df_de_c = df_de_c.loc[:, top_genes]
df_de_c

,,ADI1,IKZF1,CORO1A,TSTD1,PFN1,ACAP1,RPS16,RASAL3,PSMD6,MYL12A,...,PTPRC,RPS15A,EIF1,CACYBP,SPAG7,CHCHD10,RNASET2,TXNL1,MYO16,ZFAND2A
cell_type,sm_name,,,,,,,,,,,,,,,,,,,,,
NK cells,Mometasone Furoate,-0.896645,9.806931,-3.563557,-4.473388,-2.881752,-0.139482,0.010457,-0.662315,-0.109169,-0.299154,...,-4.894433,-0.175827,-0.035167,-0.177853,-1.522618,-0.454055,-0.600477,0.588757,3.317743,0.251185
T cells CD4+,Mometasone Furoate,-0.951883,2.301396,-5.079463,-0.349467,-5.162080,-3.618755,0.146456,-3.804370,0.037972,-0.484659,...,-8.196573,0.094622,0.156551,-0.848501,-0.879017,-0.764640,8.082415,0.284501,0.704031,0.651318
T cells CD8+,Mometasone Furoate,-0.682107,1.154251,-0.519687,-1.002865,-0.325118,-0.838742,-0.233849,0.361345,-0.124355,0.025818,...,-3.774185,-0.295623,-0.204719,0.081561,-1.547187,-0.451052,0.238005,0.304270,0.316833,1.759029
T regulatory cells,Mometasone Furoate,-1.862848,-1.787683,-0.938528,-0.524443,-2.664869,-0.226876,-0.184264,-0.034959,-0.414781,-0.172683,...,-1.045583,-0.109590,0.007464,-0.556263,-0.186470,-0.327810,-0.021450,-0.120693,1.298248,-0.361239
NK cells,Bosutinib,-1.564884,0.308813,-1.334254,-1.350785,-0.324929,0.025249,0.067056,-0.428556,0.802223,-0.184578,...,-0.262809,0.177862,0.575707,-0.445198,-0.380964,-0.584042,1.333522,0.200857,1.677002,0.771536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T regulatory cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6-yl)pyrimidin-2-amine,0.271916,0.290611,-2.446522,-0.903197,-4.537458,-0.190343,0.088412,-0.005610,-1.326677,-3.518237,...,0.133736,0.058746,1.029651,-1.663079,-1.449261,-0.050182,-0.130186,-0.228239,2.604641,0.224477
NK cells,CGM-097,0.048247,-2.007003,-0.356076,-1.137688,-2.015644,0.183110,-0.384835,0.187911,-0.465478,-0.813424,...,-1.062947,-0.216339,-0.241669,-0.315077,-0.027017,-0.220823,-0.323535,-0.414019,6.149201,-1.024855
T cells CD4+,CGM-097,-0.555006,-1.450865,-1.075604,-1.349872,-2.687981,-1.008319,-0.158965,-0.622928,-1.645391,-1.278407,...,-1.377505,-0.027173,-0.302803,-1.331745,-0.604094,-0.605465,-1.687392,0.006919,2.214352,-0.382118


In [13]:
df_lfc_c = df_lfc.loc[df_de_c.index, df_de_c.columns]
df_lfc_c.shape

(256, 256)

In [14]:
enhanced_predictions = []
for i, d in enumerate(top_drugs):
    print(i, d)
    scm = scape.model.create_default_model(n_genes, df_de_c, df_lfc_c)
    result = scm.train(
        val_cells=[cell], 
        val_drugs=[d],
        input_columns=top_genes,
        epochs=800,
        output_folder="_models",
        config_file_name="enhanced_config.pkl",
        model_file_name=f"enhanced_drug{i}.keras",
        baselines=["zero", "slogpval_drug"],
    )
    # Collect prediction in the OOF data
    df_pred = scm.predict(df_sub_ix)
    enhanced_predictions.append(df_pred)

df_sub_enhanced = pd.DataFrame(np.median(enhanced_predictions, axis=0), index=df_sub_ix.index, columns=df_de_c.columns)
df_sub_enhanced


0 IKK Inhibitor VII
Baseline zero MRRMSE: 0.9092
Baseline slogpval_drug MRRMSE: 0.8098
Model size (MB): 2.11
Model inputs: ['in_slogpval_drug', 'in_lfc_drug', 'in_slogpval_cell_encoder', 'in_lfc_cell_encoder', 'in_slogpval_cell_decoder', 'in_lfc_cell_decoder']
Model will be saved to _models\enhanced_drug0.keras
Epoch    1: val_loss None -> 1.8480, train loss: 5.1533 (lr=0.00500)
	 > Improvement factor/percentage zero                 (err=0.9092): 0.4920, -103.26%
	 > Improvement factor/percentage slogpval_drug        (err=0.8098): 0.4382, -128.20%
Epoch   28: val_loss 1.8480 -> 1.5584, train loss: 3.6023 (lr=0.00500)
	 > Improvement factor/percentage zero                 (err=0.9092): 0.5834, -71.41%
	 > Improvement factor/percentage slogpval_drug        (err=0.8098): 0.5197, -92.44%
Epoch   68: val_loss 1.5584 -> 1.4982, train loss: 2.9521 (lr=0.00500)
	 > Improvement factor/percentage zero                 (err=0.9092): 0.6068, -64.79%
	 > Improvement factor/percentage slogpval_drug  

ADI1  \
cell_type     sm_name                                                         
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...   0.849172   
              ABT-199 (GDC-0199)                                        NaN   
              ABT737                                               0.037760   
              AMD-070 (hydrochloride)                                   NaN   
              AT 7867                                                   NaN   
...                                                                     ...   
Myeloid cells Vandetanib                                                NaN   
              Vanoxerine                                                NaN   
              Vardenafil                                                NaN   
              Vorinostat                                         -10.515562   
              YK 4-279                                                  NaN   

                                                                     IKZF1  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.479185   
              ABT-199 (GDC-0199)                                       NaN   
              ABT737                                             -0.367102   
              AMD-070 (hydrochloride)                                  NaN   
              AT 7867                                                  NaN   
...                                                                    ...   
Myeloid cells Vandetanib                                               NaN   
              Vanoxerine                                               NaN   
              Vardenafil                                               NaN   
              Vorinostat                                         -2.924438   
              YK 4-279                                                 NaN   

                                                                    CORO1A  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6... -4.160229   
              ABT-199 (GDC-0199)                                       NaN   
              ABT737                                             -0.804386   
              AMD-070 (hydrochloride)                                  NaN   
              AT 7867                                                  NaN   
...                                                                    ...   
Myeloid cells Vandetanib                                               NaN   
              Vanoxerine                                               NaN   
              Vardenafil                                               NaN   
              Vorinostat                                         -0.306481   
              YK 4-279                                                 NaN   

                                                                     TSTD1  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.203586   
              ABT-199 (GDC-0199)                                       NaN   
              ABT737                                             -0.542970   
              AMD-070 (hydrochloride)                                  NaN   
              AT 7867                                                  NaN   
...                                                                    ...   
Myeloid cells Vandetanib                                               NaN   
              Vanoxerine                                               NaN   
              Vardenafil                                               NaN   
              Vorinostat                                          5.111400   
              YK 4-279                                                 NaN   

In [15]:
df_focus = df_sub.copy()
df_focus.update(df_sub_enhanced)
df_focus

A1BG  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.032278   
              ABT-199 (GDC-0199)                                 -0.019174   
              ABT737                                              0.323903   
              AMD-070 (hydrochloride)                            -0.099841   
              AT 7867                                             0.099627   
...                                                                    ...   
Myeloid cells Vandetanib                                          0.021890   
              Vanoxerine                                          0.295531   
              Vardenafil                                          0.074475   
              Vorinostat                                          0.505049   
              YK 4-279                                            0.068411   

                                                                  A1BG-AS1  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.151656   
              ABT-199 (GDC-0199)                                  0.026636   
              ABT737                                              0.065766   
              AMD-070 (hydrochloride)                             0.122193   
              AT 7867                                             0.069439   
...                                                                    ...   
Myeloid cells Vandetanib                                         -0.065695   
              Vanoxerine                                         -0.084059   
              Vardenafil                                          0.029403   
              Vorinostat                                          0.182648   
              YK 4-279                                           -0.107688   

                                                                       A2M  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.387460   
              ABT-199 (GDC-0199)                                 -0.013851   
              ABT737                                              0.088764   
              AMD-070 (hydrochloride)                             0.034077   
              AT 7867                                             0.160960   
...                                                                    ...   
Myeloid cells Vandetanib                                         -0.664498   
              Vanoxerine                                         -0.424479   
              Vardenafil                                         -0.055442   
              Vorinostat                                         -2.302280   
              YK 4-279                                           -0.798130   

                                                                   A2M-AS1  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.837589   
              ABT-199 (GDC-0199)                                 -0.032553   
              ABT737                                              0.297251   
              AMD-070 (hydrochloride)                             0.042597   
              AT 7867                                             0.343156   
...                                                                    ...   
Myeloid cells Vandetanib                                         -0.637244   
              Vanoxerine                                         -0.364767   
              Vardenafil                                         -0.303477   
              Vorinostat                                         -0.449509   
              YK 4-279                                           -0.770061   

          

In [16]:
df_submission = 0.80 * df_focus + 0.20 * df_sub
df_submission

A1BG  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.032278   
              ABT-199 (GDC-0199)                                 -0.019174   
              ABT737                                              0.323903   
              AMD-070 (hydrochloride)                            -0.099841   
              AT 7867                                             0.099627   
...                                                                    ...   
Myeloid cells Vandetanib                                          0.021890   
              Vanoxerine                                          0.295531   
              Vardenafil                                          0.074475   
              Vorinostat                                          0.505049   
              YK 4-279                                            0.068411   

                                                                  A1BG-AS1  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.151656   
              ABT-199 (GDC-0199)                                  0.026636   
              ABT737                                              0.065766   
              AMD-070 (hydrochloride)                             0.122193   
              AT 7867                                             0.069439   
...                                                                    ...   
Myeloid cells Vandetanib                                         -0.065695   
              Vanoxerine                                         -0.084059   
              Vardenafil                                          0.029403   
              Vorinostat                                          0.182648   
              YK 4-279                                           -0.107688   

                                                                       A2M  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.387460   
              ABT-199 (GDC-0199)                                 -0.013851   
              ABT737                                              0.088764   
              AMD-070 (hydrochloride)                             0.034077   
              AT 7867                                             0.160960   
...                                                                    ...   
Myeloid cells Vandetanib                                         -0.664498   
              Vanoxerine                                         -0.424479   
              Vardenafil                                         -0.055442   
              Vorinostat                                         -2.302280   
              YK 4-279                                           -0.798130   

                                                                   A2M-AS1  \
cell_type     sm_name                                                        
B cells       5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6...  0.837589   
              ABT-199 (GDC-0199)                                 -0.032553   
              ABT737                                              0.297251   
              AMD-070 (hydrochloride)                             0.042597   
              AT 7867                                             0.343156   
...                                                                    ...   
Myeloid cells Vandetanib                                         -0.637244   
              Vanoxerine                                         -0.364767   
              Vardenafil                                         -0.303477   
              Vorinostat                                         -0.449509   
              YK 4-279                                           -0.770062   

          